## Importações



In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import pickle

In [2]:
# Assuming the file is named 'dataframe.pkl'
with open('dataframe.pkl', 'rb') as f:
    df = pickle.load(f)

In [3]:
df.keys()

Index(['mof', 'void fraction', 'spacegroup_num', 'crystal_system',
       'crystal_system_int', 'is_centrosymmetric', 'n_symmetry_ops', 'Zn', 'H',
       'C', 'O', 'F', 'Cl', 'Br', 'N'],
      dtype='object')

In [4]:
df

,mof,void fraction,spacegroup_num,crystal_system,crystal_system_int,is_centrosymmetric,n_symmetry_ops,Zn,H,C,O,F,Cl,Br,N
0,hMOF-6,0.754903,1,triclinic,7,False,1,4.0,6.0,24.0,13.0,6.0,0.0,0.0,0.0
1,hMOF-0,0.795539,160,trigonal,3,False,6,4.0,12.0,24.0,13.0,0.0,0.0,0.0,0.0
2,hMOF-7,0.327134,1,triclinic,7,False,2,8.0,12.0,48.0,26.0,12.0,0.0,0.0,0.0
3,hMOF-5,0.169003,1,triclinic,7,False,3,12.0,18.0,72.0,39.0,18.0,0.0,0.0,0.0
4,hMOF-3,0.755062,1,triclinic,7,False,1,4.0,6.0,24.0,13.0,6.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,hMOF-993,0.754829,1,triclinic,7,False,1,4.0,18.0,25.0,13.0,0.0,0.0,0.0,0.0
996,hMOF-996,0.347210,1,triclinic,7,False,2,8.0,12.0,34.0,26.0,0.0,0.0,0.0,0.0
997,hMOF-997,0.775003,1,triclinic,7,False,1,4.0,12.0,22.0,13.0,0.0,0.0,0.0,0.0
998,hMOF-999,0.753542,1,triclinic,7,False,1,4.0,16.0,22.0,13.0,0.0,0.0,0.0,0.0


## Código e discussão



### Divisão treino-teste

In [5]:
TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 61455
FEATURES = ['crystal_system_int', 'n_symmetry_ops', 'Zn', 'H', 'C', 'O', 'F', 'Cl', 'Br', 'N']
TARGET = ['void fraction']

indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

X_treino = df_treino.reindex(FEATURES, axis=1)
y_treino = df_treino.reindex(TARGET, axis=1)
X_teste = df_teste.reindex(FEATURES, axis=1)
y_teste = df_teste.reindex(TARGET, axis=1)

In [6]:
normalizador_x = MinMaxScaler()
normalizador_y = MinMaxScaler()

normalizador_x.fit(X_treino)
normalizador_y.fit(y_treino)

X_treino = normalizador_x.transform(X_treino)
y_treino = normalizador_y.transform(y_treino)
X_teste = normalizador_x.transform(X_teste)
y_teste = normalizador_y.transform(y_teste)

In [7]:
X_treino = torch.tensor(X_treino, dtype=torch.float32)
y_treino = torch.tensor(y_treino, dtype=torch.float32)
X_teste = torch.tensor(X_teste, dtype=torch.float32)
y_teste = torch.tensor(y_teste, dtype=torch.float32)

In [8]:
class MLP(nn.Module):
    def __init__(
        self, num_dados_entrada, neuronios_c1, neuronios_c2, neuronios_c3, neuronios_c4, num_targets
    ):
        # Temos que inicializar a classe mãe
        super().__init__()

        # Definindo as camadas da rede
        self.camadas = nn.Sequential(
                    nn.Linear(num_dados_entrada, neuronios_c1),
                    nn.ReLU(),
                    nn.Linear(neuronios_c1, neuronios_c2),
                    nn.ReLU(),
                    nn.Linear(neuronios_c2, neuronios_c3),
                    nn.ReLU(),
                    nn.Linear(neuronios_c3, neuronios_c4),
                    nn.ReLU(),
                    nn.Linear(neuronios_c4, num_targets),
                )

    def forward(self, x):
        """Esse é o método que executa a rede do pytorch."""
        x = self.camadas(x)
        return x

In [9]:
NUM_DADOS_DE_ENTRADA = X_treino.shape[1]
NUM_DADOS_DE_SAIDA = y_treino.shape[1]
NEURONIOS_C1 = 150
NEURONIOS_C2 = 100
NEURONIOS_C3 = 80
NEURONIOS_C4 = 60

minha_MLP = MLP(NUM_DADOS_DE_ENTRADA, NEURONIOS_C1, NEURONIOS_C2, NEURONIOS_C3, NEURONIOS_C4, NUM_DADOS_DE_SAIDA)

In [10]:
y_prev = minha_MLP(X_treino)

In [11]:
TAXA_DE_APRENDIZADO = 0.0005

# função perda será o erro quadrático médio
fn_perda = nn.MSELoss()

# otimizador será o Adam, um tipo de descida do gradiente
otimizador = optim.Adam(minha_MLP.parameters(), lr=TAXA_DE_APRENDIZADO)

In [12]:
minha_MLP.train()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=10, out_features=150, bias=True)
    (1): ReLU()
    (2): Linear(in_features=150, out_features=100, bias=True)
    (3): ReLU()
    (4): Linear(in_features=100, out_features=80, bias=True)
    (5): ReLU()
    (6): Linear(in_features=80, out_features=60, bias=True)
    (7): ReLU()
    (8): Linear(in_features=60, out_features=1, bias=True)
  )
)

In [13]:
NUM_EPOCAS = 2000

y_true = y_treino

for epoca in range(NUM_EPOCAS):
    # forward pass
    y_pred = minha_MLP(X_treino)

    # zero grad
    otimizador.zero_grad()

    # loss
    loss = fn_perda(y_pred, y_true)

    # backpropagation
    loss.backward()

    # atualiza parâmetros
    otimizador.step()

    # mostra resultado
    print(epoca, loss.data)

0 tensor(0.4192)
1 tensor(0.4061)
2 tensor(0.3941)
3 tensor(0.3828)
4 tensor(0.3716)
5 tensor(0.3605)
6 tensor(0.3493)
7 tensor(0.3380)
8 tensor(0.3262)
9 tensor(0.3140)
10 tensor(0.3012)
11 tensor(0.2882)
12 tensor(0.2751)
13 tensor(0.2615)
14 tensor(0.2472)
15 tensor(0.2322)
16 tensor(0.2164)
17 tensor(0.1998)
18 tensor(0.1827)
19 tensor(0.1657)
20 tensor(0.1486)
21 tensor(0.1317)
22 tensor(0.1153)
23 tensor(0.0997)
24 tensor(0.0853)
25 tensor(0.0728)
26 tensor(0.0624)
27 tensor(0.0547)
28 tensor(0.0498)
29 tensor(0.0480)
30 tensor(0.0489)
31 tensor(0.0519)
32 tensor(0.0558)
33 tensor(0.0595)
34 tensor(0.0618)
35 tensor(0.0622)
36 tensor(0.0608)
37 tensor(0.0580)
38 tensor(0.0544)
39 tensor(0.0505)
40 tensor(0.0469)
41 tensor(0.0439)
42 tensor(0.0418)
43 tensor(0.0404)
44 tensor(0.0397)
45 tensor(0.0395)
46 tensor(0.0396)
47 tensor(0.0398)
48 tensor(0.0400)
49 tensor(0.0400)
50 tensor(0.0397)
51 tensor(0.0392)
52 tensor(0.0385)
53 tensor(0.0376)
54 tensor(0.0366)
55 tensor(0.0356)
56

In [14]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_treino)
    y_pred = minha_MLP(X_treino)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[0.803664] [0.80144751]
[0.721097] [0.69020249]
[0.27111701] [0.22746626]
[0.84661497] [0.87951369]
[0.77047001] [0.76399146]
[0.223383] [0.27800969]
[0.68672899] [0.71138116]
[0.78892098] [0.73169772]
[0.53113902] [0.53234284]
[0.84562299] [0.87285487]
[0.87955599] [0.86919469]
[0.78057098] [0.7668414]
[0.30288899] [0.28872643]
[0.86152498] [0.868894]
[0.82001001] [0.85161909]
[0.66264] [0.65368606]
[0.42030401] [0.58588232]
[0.89789798] [0.87666568]
[0.77956401] [0.70187057]
[0.52906397] [0.54381916]
[0.32713399] [0.31330903]
[0.85649399] [0.84703571]
[0.311915] [0.31330903]
[0.52430498] [0.52186471]
[0.48495801] [0.58588232]
[0.811945] [0.82066282]
[0.69132702] [0.68844274]
[0.725891] [0.69830539]
[0.692027] [0.73810533]
[0.72611201] [0.68844274]
[0.50893901] [0.54467639]
[0.62359101] [0.60843765]
[0.83282197] [0.82115827]
[0.737951] [0.71678998]
[0.65692202] [0.69745932]
[0.84609802] [0.84315192]
[0.52490598] [0.57055251]
[0.80258703] [0.7756686]
[0.725514] [0.70921416]
[0.230074] 

In [15]:
minha_MLP.eval()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=10, out_features=150, bias=True)
    (1): ReLU()
    (2): Linear(in_features=150, out_features=100, bias=True)
    (3): ReLU()
    (4): Linear(in_features=100, out_features=80, bias=True)
    (5): ReLU()
    (6): Linear(in_features=80, out_features=60, bias=True)
    (7): ReLU()
    (8): Linear(in_features=60, out_features=1, bias=True)
  )
)

In [16]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_teste)
    y_pred = minha_MLP(X_teste)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[0.81500402] [0.85227782]
[0.54636399] [0.54005718]
[0.320884] [0.24918678]
[0.770908] [0.78339317]
[0.337309] [0.33809344]
[0.85877899] [0.84702327]
[0.706279] [0.70590029]
[0.696503] [0.71138116]
[0.34748901] [0.35434045]
[0.84756002] [0.83997114]
[0.70656499] [0.68809178]
[0.87377498] [0.89896522]
[0.55245498] [0.6102381]
[0.41821299] [0.38839621]
[0.68829798] [0.72282026]
[0.666691] [0.66847481]
[0.71481701] [0.69915272]
[0.88304901] [0.86741896]
[0.89688597] [0.8917257]
[0.59309502] [0.70427379]
[0.36248199] [0.33769742]
[0.71725401] [0.7325962]
[0.829012] [0.82207854]
[0.791027] [0.74587999]
[0.73646402] [0.70686584]
[0.267539] [0.34205041]
[0.74453998] [0.75048356]
[0.90191099] [0.70427379]
[0.76071502] [0.75842549]
[0.88085603] [0.69933531]
[0.87084098] [0.85472926]
[0.72560999] [0.74162318]
[0.272509] [0.25204679]
[0.56935001] [0.51949892]
[0.31756999] [0.2984567]
[0.216026] [0.26914028]
[0.81453501] [0.7926567]
[0.760401] [0.79013819]
[0.67104502] [0.66432422]
[0.48551499] [0

In [17]:
X_teste[0]

tensor([1.0000, 0.0000, 0.0000, 0.0546, 0.2159, 0.0000, 0.0000, 0.0000, 0.0000,
        0.2344])

In [18]:
RMSE = mean_squared_error(y_true, y_pred, squared=False)
print(f'Loss do teste: {RMSE}')

Loss do teste: 0.0616200315100898


In [19]:
import random
from funcoes import populacao_cnb
from funcoes import selecao_torneio_min as funcao_selecao # esse já temos!
from funcoes import cruzamento_ponto_simples as funcao_cruzamento
from funcoes import mutacao_cnb

In [20]:
# Constantes de busca

TAMANHO_POP = 12 # quantidade de indivíduos
NUM_GERACOES = 10000 # número de gerações
CHANCE_CRUZAMENTO = 0.5 # chance de ocorrer o cruzamento entre dois indivíduos
CHANCE_MUTACAO = 0.02 # chance de ocorrer mutação em cada indivíduo durante cada geração

# Constantes de problema
QUANTIDADE_MAX_ATOMOS = 100 # quantidade de valor máximo que um gene pode assumir
NUM_ELEMENTOS = 10 # quantidade de genes presentes em cada indivíduo
TAMANHO_PORO_DESEJADO = 0.3

In [21]:
# Funções Locais

def cria_populacao_inicial(tamanho, numero_genes):
    return populacao_cnb(tamanho, numero_genes, VALOR_MAX_CAIXA)

def funcao_mutacao(individuo):
    return mutacao_cnb(individuo, VALOR_MAX_CAIXA)

def funcao_objetivo(individuo, TAMANHO_PORO_DESEJADO):
    
    y_pred = minha_MLP(individuo_X)
    y_pred = normalizador_y.inverse_transform(y_pred)
    return abs(y_pred - TAMANHO_PORO_DESEJADO)

In [22]:
populacao = cria_populacao_inicial(TAMANHO_POP, NUM_GENES) # cria aleatoriamente uma população inicial

print('População inicial:') # mostra qual foi a população criada aleatoriamente
for i, ind in enumerate(populacao):
    print('Individuo ', i+1, ': ', ind)

for _ in range(NUM_GERACOES): # loop que começa a rodar cada geração
    fitness = funcao_objetivo_pop(populacao) # cálculo da função objetivo de cada indivíduo da população
    populacao = funcao_selecao(populacao, fitness) # seleção de roleta com diferentes pesos, baseados na função fitness
    
    pais = populacao[0::2] # definição dos indivíduos que serão pais
    maes = populacao[1::2] # definição dos indivíduos que serão mães
    contador = 0 # estratégia para colocar os filhos no lugar dos pais
    for pai, mae in zip(pais, maes): # laço de repetição para pegar itens da lista de pais e mães
        if random.random() < CHANCE_CRUZAMENTO: # aplicando a possibilidade de cruzamento
            # vai acertar o cruzamento
            filho1, filho2 = funcao_cruzamento(pai, mae) # "calculando" o filho 1 e o filho 2
            populacao[contador] = filho1 # trocando o pai pelo filho 1
            populacao[contador + 1] = filho2 # trocando a mãe pelo filho 2
            
        contador = contador + 2 # atualização do contador
    
    for n in range(len(populacao)): #laço de repetição para mutação
        if random.random() <= CHANCE_MUTACAO: # chance de mutação
            individuo = populacao[n] # esxolhe o indivíduo
            populacao[n] = funcao_mutacao(individuo) # muta o indivíduo
        
    
print()
print('População final:') # mostra qual foi a população final selecionada geneticamente
for i, ind in enumerate(populacao):
    print('Individuo ', i+1, ': ', ind)

NameError: name 'NUM_GENES' is not defined